In [34]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Домашнее задание к лекции "Основы веб-скрапинга"
Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [35]:
def get_habr(list1, url='https://habr.com/ru/search/'):
   habr = pd.DataFrame()
   URL = url
   for i in list1:
     query = '+'.join(i.split())
     params = {
        'q': query,
        'target_type': 'posts',
        'order': 'relevance'
          }
     req = requests.get(URL, params=params)
     soup= BeautifulSoup(req.text)
     articles = soup.find_all('div', class_='tm-article-snippet')
     for article in articles:
       title = article.find('h2').text
       link = article.find('a', 'tm-title__link').get('href')
       date = pd.to_datetime(article.find('time').get('datetime'))
       row = {'дата':date, 'заголовок':title, 'link':link}
       habr = pd.concat([habr, pd.DataFrame([row])])
   return habr.drop_duplicates().reset_index(drop=True)

get_habr(['python', 'analyze']).tail(3)

,дата,заголовок,link
37,2019-06-24 07:01:14+00:00,How to quickly check out interesting warnings ...,/ru/companies/pvs-studio/articles/457330/
38,2020-10-12 13:26:57+00:00,Upsetting Opinions about Static Analyzers,/ru/companies/pvs-studio/articles/523114/
39,2008-12-29 08:54:41+00:00,Mono Migration Analyzer (MoMA),/ru/articles/48027/


In [39]:
def get_habr_pages(list1, pages=4, url='https://habr.com/ru/search/'):
    for page in range(1, pages+1):
        if page==1:
            df1 = get_habr(list1, url=url)
        if page>1:
            df2 = get_habr(list1, url=f'{url}page{page}/')
            df1 = pd.concat([df1, df2], ignore_index=True)
    return df1.drop_duplicates().reset_index(drop=True)

get_habr_pages(['python', 'analyze']).tail(3)

,дата,заголовок,link
152,2020-02-13 11:42:06+00:00,PVS-Studio Is Now in Chocolatey: Checking Choc...,/ru/companies/pvs-studio/articles/488204/
153,2007-09-12 13:59:35+00:00,«Ашманов и партнёры» запустили анализатор поис...,/ru/articles/14024/
154,2017-02-01 07:00:06+00:00,Как писать кривые запросы с неоптимальным план...,/ru/articles/320916/
